# Groww auth\n\n1) Copy `.secrets.example.toml` to `.secrets.toml` and paste your keys.\n2) Run the cells to fetch an access token.\n\nNote: Avoid sharing or committing tokens/secrets.

In [ ]:
from pathlib import Path
import os
import tomllib

from growwapi import GrowwAPI

secrets_path = Path('.secrets.toml')
cfg = {}
if secrets_path.exists():
    cfg = tomllib.loads(secrets_path.read_text())

approval_api_key = cfg.get('approval_api_key') or cfg.get('api_key') or os.environ.get('GROWW_APPROVAL_API_KEY') or os.environ.get('GROWW_API_KEY')
approval_secret = cfg.get('approval_secret') or cfg.get('secret') or os.environ.get('GROWW_APPROVAL_SECRET') or os.environ.get('GROWW_API_SECRET')

totp_token = cfg.get('totp_token') or os.environ.get('GROWW_TOTP_TOKEN')
totp = cfg.get('totp') or os.environ.get('GROWW_TOTP')
totp_secret = cfg.get('totp_secret') or os.environ.get('GROWW_TOTP_SECRET')

use_totp = bool(totp_token and (totp or totp_secret))
if use_totp:
    if not totp and totp_secret:
        import pyotp
        totp = pyotp.TOTP(totp_secret).now()
    access_token = GrowwAPI.get_access_token(api_key=totp_token, totp=totp)
else:
    access_token = GrowwAPI.get_access_token(api_key=approval_api_key, secret=approval_secret)

groww = GrowwAPI(access_token)

preview = str(access_token)
preview = preview[:6] + '...' + preview[-4:] if len(preview) > 12 else '***'
preview